Hidden cell with Google and Forsquare keys:

In [12]:
# The code was removed by Watson Studio for sharing.

Required code from part 1 and part 2 of the assignement (required for this part). Please ignore.

In [13]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table')
td1 = table.find_all('td',{'style':"width:11%; vertical-align:top;"})
td2 = table.find_all('td',{'style':"vertical-align:top;"})
table_black=td1+td2
table_str=str(table_black)
table_clr = ''.join(map(str, table_black))
table_str=str(table_clr)
table_sp = BeautifulSoup(str(table_str),'html.parser')
cleared=[]
for line in table_sp.findAll("td"):
    line_str=str(line)
    i=0
    if "(" not in line_str:
        line_str = line_str + "(added"+"str(i)"+")"
        i = i + 1
        cleared.append(line_str)
regex = re.compile(".*?\((.*?)\)")
table_txt = BeautifulSoup(table_str,'html.parser').getText(' ', strip=True)
neighb = re.findall(regex, table_txt)
m_and_b_temp = re.sub("[\(\[].*?[\)\]]", "", table_str)
m_and_b = BeautifulSoup(str(m_and_b_temp),'html.parser')
postal_c_s=m_and_b.findAll('p')
postal_c = BeautifulSoup(str(postal_c_s),'html.parser')
postal=[]
for n in postal_c.findAll("b"):
    postal.append(n.getText())
boroughs = []
for line in m_and_b.findAll("td"):
    for b in line.findAll("span"):
        head, sep, tail = b.getText(' ', strip=True).partition('\"') #remove html leftover
        cell = head.replace("Queen's Park\n","Queen's Park").replace("Mississauga Canada Post Gateway Processing Centre", "Mississauga").replace("East Toronto Business reply mail Processing Centre 969 Eastern", "East Toronto").replace("Downtown Toronto Stn A PO Boxes 25 The Esplanade","Downtown Toronto").replace("EtobicokeNorthwest","Etobicoke Northwest")
        boroughs.append(cell)
del neighb[12]
del neighb[12]
neighb.insert(12, "Don Mills / Flemingdon Park")
neighb.insert(4, "Queen's Park")
del neighb[41]
del neighb[41]
neighb.insert(41, "Downsview / CFB Toronto")
neighb = [word.replace(' / ',', ') for word in neighb]

df = pd.DataFrame(
    {'PostalCode': postal,
     'Borough': boroughs,
     'Neighborhood': neighb
    })

rows = [7, 13, 40,46, 53, 59, 60,72,94]
i=0
for row in rows:
    r_str= str(df.loc[row, 'Borough'])
    df.loc[row, 'Neighborhood'] = df.loc[row, 'Neighborhood'] +" "+ r_str.rsplit(" ",1)[1]
    df.loc[row, 'Borough'] = r_str.rsplit(" ",1)[0]

df = df[['PostalCode','Borough','Neighborhood']]

latitude =[]
longitude=[]
i=0
for post in postal:
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, 'toronto'+post) # Added Toronto to address for better results
    response = requests.get(url).json()
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude.append(geographical_data['lat'])
    longitude.append(geographical_data['lng'])
    
lat_ser = pd.Series(latitude)
lon_ser = pd.Series(longitude)

df['Latitude'] = lat_ser.values
df['Longitude'] = lon_ser.values

An at-a-glance visual representation of Toronto neighborhoods:

In [55]:
import folium

# create map of Toronto using latitude and longitude values
latitude_tor = 43.70011
longitude_tor = -79.4163

map_toronto = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(latitude, longitude, boroughs, neighb):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let us select only the boroughs that include the name Toronto:

In [42]:
df_s = df[df['Borough'].str.contains('Toronto')]
df_s

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,M4J,East York East Toronto,The Danforth East,43.685347,-79.338106


Set some variables to use later:

In [43]:
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

We are borrowing the get_category_type function from the Foursquare lab. We will use it to get information from the "items" key.

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

The function to get the data from Forsquare for all the Toronto neighbourhoods:

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

Run the above function:

In [46]:
toronto_venues = getNearbyVenues(names=df_s['Neighborhood'],
                                   latitudes=df_s['Latitude'],
                                   longitudes=df_s['Longitude']
                                  )
toronto_venues.head()

 Regent Park, Harbourfront 
 Garden District , Ryerson 
 St. James Town 
 The Beaches 
Berczy Park
Central Bay Street 
Christie
 Richmond, Adelaide, King 
 Dufferin, Dovercourt Village 
 The Danforth East
 Harbourfront East, Union Station, Toronto Islands 
 Little Portugal, Trinity 
Downsview, CFB Toronto
 Toronto Dominion Centre, Design Exchange 
 Brockton, Parkdale Village, Exhibition Place 
 India Bazaar, The Beaches West
 Commerce Court, Victoria Hotel 
Studio District
 Lawrence Park 
Roselawn
 Davisville North
 Forest Hill North & West
 High Park, The Junction South
 North Toronto West
 The Annex, North Midtown, Yorkville 
 Parkdale, Roncesvalles 
 Davisville 
 University of Toronto, Harbord
 Runnymede, Swansea 
 Moore Park, Summerhill East
 Kensington Market, Chinatown, Grange Park 
 Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park 
 CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport 
 Rosedale 
Enclave of M

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Ha...",0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,Church and Wellesley,0.011765,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.011765,0.011765,0.011765,0.00,0.000000,0.011765,0.00
3,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.00
4,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.027027,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
7,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.00
8,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.25000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
9,"Garden District , Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.010000,0.000000,0.010000,0.00,0.010000,0.000000,0.00


Let's print each neighborhood along with the top 5 most common venues:

In [48]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Brockton, Parkdale Village, Exhibition Place ----
            venue  freq
0     Coffee Shop  0.14
1  Breakfast Spot  0.09
2            Café  0.09
3             Gym  0.05
4   Burrito Place  0.05


---- CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport ----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3  Sculpture Garden  0.07
4             Plane  0.07


---- Church and Wellesley ----
                 venue  freq
0  Japanese Restaurant  0.06
1          Coffee Shop  0.05
2              Gay Bar  0.05
3     Sushi Restaurant  0.05
4         Burger Joint  0.05


---- Commerce Court, Victoria Hotel ----
                 venue  freq
0          Coffee Shop  0.13
1                 Café  0.06
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


---- Davisville ----
                venue  freq
0      Sandwich Place  0.08
1        Dessert Shop  

Put the top 10 most common values in the panda dataframe:

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store,Convenience Store,Climbing Gym,Nightclub,Caribbean Restaurant,Burrito Place
1,"CN Tower, King and Spadina, Railway Lands, Ha...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport
2,Church and Wellesley,Japanese Restaurant,Burger Joint,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Bubble Tea Shop,Nightclub,Pub
3,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Deli / Bodega,Bakery
4,Davisville,Sandwich Place,Dessert Shop,Thai Restaurant,Coffee Shop,Gym,Sushi Restaurant,Italian Restaurant,Café,Pizza Place,Seafood Restaurant
5,Davisville North,Clothing Store,Park,Food & Drink Shop,Burger Joint,Hotel,Breakfast Spot,Sandwich Place,Playground,Grocery Store,Eastern European Restaurant
6,"Dufferin, Dovercourt Village",Bakery,Liquor Store,Supermarket,Pharmacy,Gym / Fitness Center,Music Venue,Discount Store,Café,Bus Stop,Middle Eastern Restaurant
7,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Burger Joint,Deli / Bodega,Concert Hall
8,Forest Hill North & West,Trail,Sushi Restaurant,Jewelry Store,Park,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
9,"Garden District , Ryerson",Clothing Store,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Pizza Place,Burger Joint,Plaza,Bubble Tea Shop


Run *k*-means to cluster the neighborhood into 5 clusters:

In [50]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [51]:
toronto_merged = df_s.copy() # Added copy to avoid a warning

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Park,Mexican Restaurant,Breakfast Spot,Pub,Theater,Health Food Store,Bank
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Pizza Place,Burger Joint,Plaza,Bubble Tea Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Clothing Store,Hotel,Restaurant,Cosmetics Shop,Italian Restaurant,Gastropub,Japanese Restaurant,Cocktail Bar
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Boutique,Pub,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Café,Cheese Shop,Restaurant,Steakhouse


Let's visualize the resulting clusters:

In [54]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters